In [1]:
import gensim.downloader as api
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle

import os
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import constants

#get training path
train_dir = constants.TRAIN_DIR
clean_dir = constants.CLEAN_DIR
art_prefix = constants.Text_Prefix
img_prefix = constants.Media_Prefix
#tag types
tag_types = ['org', 'place', 'subject','person']
# article_summary = pd.read_csv(f'{train_dir}/{art_prefix}summary.csv')
article_summary = pd.read_csv(f'{constants.CLEAN_DIR}/article_summary.csv')
image_summary =  pd.read_csv(f'{train_dir}/{img_prefix}summary.csv')
#preview dir
preview_dir = f'{constants.DATA_DIR}/preview'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.nlp_util.easynlp import easynlp

Building prefix dict from /Users/pwong/anaconda3/lib/python3.6/site-packages/synonyms/data/vocab.txt ...
I1202 15:56:55.021873 4645250496 __init__.py:111] Building prefix dict from /Users/pwong/anaconda3/lib/python3.6/site-packages/synonyms/data/vocab.txt ...
Loading model from cache /var/folders/wd/fnzsxwb5585gt0tyhx_7znnh0000gn/T/jieba.ub1bc640de0fe7bfc05777e6fa774082c.cache
I1202 15:56:55.037290 4645250496 __init__.py:131] Loading model from cache /var/folders/wd/fnzsxwb5585gt0tyhx_7znnh0000gn/T/jieba.ub1bc640de0fe7bfc05777e6fa774082c.cache


>> Synonyms load wordseg dict [/Users/pwong/anaconda3/lib/python3.6/site-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.723 seconds.
I1202 15:56:56.756875 4645250496 __init__.py:163] Loading model cost 1.723 seconds.
Prefix dict has been built succesfully.
I1202 15:56:56.757810 4645250496 __init__.py:164] Prefix dict has been built succesfully.


>> Synonyms on loading stopwords [/Users/pwong/anaconda3/lib/python3.6/site-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/Users/pwong/anaconda3/lib/python3.6/site-packages/synonyms/data/words.vector] ...


/Users/pwong/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
!pip install jieba

  Using cached https://files.pythonhosted.org/packages/71/46/c6f9179f73b818d5827202ad1c4a94e371a29473b7f043b736b4dab6b8cd/jieba-0.39.zip
  Stored in directory: /Users/pwong/Library/Caches/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!pip install synonyms

  Using cached https://files.pythonhosted.org/packages/3e/47/065b8f34aad7bbf34447df72e4188a5ac6ae7106af5e2f91f5a0ed33f4e0/synonyms-3.10.2.tar.gz
  Stored in directory: /Users/pwong/Library/Caches/pip/wheels/35/34/8c/195ac92995b84be9e93dcc5ddbea940a1545bbca9a919e93b5
Successfully built synonyms
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
article_summary.head()

,id,version,version_created,content_type,language,city,country,long_lat,title,headline,headline_extended,summary,full_text
0,964285bee74e430cb22441e59a968e76,1,2019-06-29T18:38:39Z,text,en,Avon,United States,"[-77.73411, 42.89547]",NY--911 Call-Probation,Sheriff: NY man calls 911 to say he's violatin...,Authorities say a western New York man has bee...,Authorities say a western New York man has bee...,"AVON, N.Y. (AP) — Authorities say a western Ne..."
1,5485474faec244b0881838c139b4ef10,5,2019-04-18T22:22:45Z,text,en,New York,United States,"[-74.00597, 40.71427]",FBN--NFL Schedule,Champion Patriots open vs. Steelers; 5 interna...,The NFL's 100th season will begin with its mos...,The NFL's 100th season will begin with its mos...,NEW YORK (AP) — The NFL's 100th season will be...
2,6aff14aef2ff4a10a4f27f4ca879ad9e,1,2019-03-12T05:00:05Z,text,en,Tokyo,Japan,"[139.69171, 35.6895]",OLY--Tokyo-500 Days,"Tick tock, tick tock: Tokyo Olympics clock hit...",The Tokyo Olympic clock has hit 500 days to go,The Tokyo Olympic clock has hit 500 days to go...,"TOKYO (AP) — Tick tock, tick tock. The Tokyo O..."
3,3a29e90a364a4ce7bb294286a8647535,1,2019-08-07T01:56:18Z,text,en,Inglewood,United States,"[-118.35313, 33.96168]",APFN-US--Inglewood Stadium-American Airlines,"American Airlines first sponsor for Rams, Char...",The new stadium at Hollywood Park being built ...,The new stadium at Hollywood Park being built ...,"INGLEWOOD, Calif. (AP) — The new stadium at Ho..."
4,bbabfcdbb53b4804adaffb2933bfc207,1,2019-05-07T13:59:07Z,text,en,Tirana,Albania,"[19.81889, 41.3275]",EU--Albania-Lions,Off to a new life: 3 Albanian lions sent to Du...,Three lions from Tirana Zoo in Albania have be...,Three formerly hungry lions being housed at Ti...,"TIRANA, Albania (AP) — Three formerly hungry l..."


In [4]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data
# >>>
# >>> result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
# >>> print("{}: {:.4f}".format(*result[0]))
# queen: 0.7699
# >>>
# >>> result = word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
# >>> print("{}: {:.4f}".format(*result[0]))
# queen: 0.8965
# >>>
# >>> print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))
# cereal
# >>>
similarity = word_vectors.similarity('woman', 'man')

In [5]:
similarity

0.8323495

In [9]:
raw_text = article_summary[article_summary.headline == 'House panel opens sweeping probe of Trump, his associates'].full_text.values[0]
raw_text

'WASHINGTON (AP) — Democrats launched\xa0a sweeping new probe of President Donald Trump, an aggressive investigation that threatens to shadow the president through the 2020 election season with potentially damaging inquiries into his White House, campaign and family businesses.House Judiciary Committee Chairman Jerrold Nadler said Monday his panel was beginning the probe into possible obstruction of justice, corruption and abuse of power and is sending document requests to 81 people linked to the president and his associates.The broad investigation could be setting the stage for an impeachment effort, although Democratic leaders have pledged to investigate all avenues and review special counsel Robert Mueller\'s upcoming report before trying any drastic action.Trump denounced the probe Tuesday, tweeting that Nadler and other Democrats "have gone stone cold CRAZY. 81 letter sent to innocent people to harass them. They won\'t get ANYTHING done for our Country!"Nadler said the document re

In [7]:
raw_text

'WASHINGTON (AP) — Democrats launched\xa0a sweeping new probe of President Donald Trump, an aggressive investigation that threatens to shadow the president through the 2020 election season with potentially damaging inquiries into his White House, campaign and family businesses.House Judiciary Committee Chairman Jerrold Nadler said Monday his panel was beginning the probe into possible obstruction of justice, corruption and abuse of power and is sending document requests to 81 people linked to the president and his associates.The broad investigation could be setting the stage for an impeachment effort, although Democratic leaders have pledged to investigate all avenues and review special counsel Robert Mueller\'s upcoming report before trying any drastic action.Trump denounced the probe Tuesday, tweeting that Nadler and other Democrats "have gone stone cold CRAZY. 81 letter sent to innocent people to harass them. They won\'t get ANYTHING done for our Country!"Nadler said the document re

In [10]:
# from easynlp import easynlp
# 获取排序的关键词排序结果写入文件
sorted_words = easynlp.get_keyword(raw_text)
print(sorted_words)

filter_all_list: [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[]
